In [8]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random2 as random
import plotly.graph_objects as go

def girvan_newman(G):
    # Make a copy of the graph so we don't modify the original
    G2 = G.copy()

    # Initialize the best modularity to be -inf
    best_modularity = -float('inf')

    while True:
        # Calculate the betweenness centrality of all edges in the graph
        edge_betweenness = nx.edge_betweenness_centrality(G2)

        # Remove the edge with the highest betweenness centrality
        max_edge = max(edge_betweenness, key=edge_betweenness.get)
        G2.remove_edge(*max_edge)

        # Calculate the modularity of the resulting communities
        communities = list(nx.connected_components(G2))
        modularity = nx.algorithms.community.modularity(G, communities)

        # If the modularity is greater than the current best, update the best modularity and best communities
        if modularity > best_modularity:
            best_modularity = modularity
            best_communities = communities

        # If there are no more edges to remove, return the best communities
        if len(G2.edges()) == 0:
            return best_communities

def main():
    # Load the Karate Club network into NetworkX
    G = nx.karate_club_graph()
    #G = nx.read_edgelist("/content/sample_data/CA-HepTh.txt")

    # Step 1: Detect communities using the Louvain method
    from networkx.algorithms.community import greedy_modularity_communities
    communities = list(greedy_modularity_communities(G))

   
    # Step 2: Select seed nodes with highest betweenness centrality in each community
    # Step 2: Select seed nodes with highest betweenness centrality in each community
    seed_nodes = []
    for comm in communities:
      node_centrality = nx.betweenness_centrality_subset(G, comm, G.nodes)
    # make a copy of the graph to remove the nodes that have already been selected as seed nodes
      G_copy = G.copy()
      G_copy.remove_nodes_from(seed_nodes)
    # get the betweenness centrality for the remaining nodes
      remaining_centrality = {node: centrality for node, centrality in node_centrality.items() if node in G_copy.nodes()}
      max_centrality = max(remaining_centrality.values())
      max_nodes = [node for node, centrality in remaining_centrality.items() if centrality == max_centrality]
      seed_node = random.choice(max_nodes)
      seed_nodes.append(seed_node)


    # Step 3: Calculate influence spread using the Linear Threshold Model
    def linear_threshold_model(G, seed_nodes, p):
        # Initialize activation probabilities and thresholds for all nodes
        activation_probabilities = {node: 0.1 for node in G.nodes()}
    ######################################*************************###############################   
        thresholds = {node: random.randint() for node in G.nodes()}

        # Initialize set of activated nodes with seed nodes
        activated_nodes = set(seed_nodes)

        # Keep activating nodes until no more nodes can be activated
        while True:
            # Determine the next set of nodes to be activated
            newly_activated = set()
            for node in G.nodes():
                if node in activated_nodes:
                    continue
                inactivated_neighbors = set(G.neighbors(node)) - activated_nodes
                neighbor_activation_probability = sum(activation_probabilities[n] for n in inactivated_neighbors)
                if neighbor_activation_probability >= thresholds[node]:
                    newly_activated.add(node)

            if not newly_activated:
                break

            # Update activation probabilities and activated nodes
            for node in newly_activated:
                activation_probabilities[node] = p
            activated_nodes |= newly_activated

        return len(activated_nodes)

    # Set the probability of activation to 0.5
    p = 0.5

    # Calculate the influence spread of the seed nodes in each community
    influence_spread = []
    for i, comm in enumerate(communities):
        # Get the seed node for this community
        seed_node = seed_nodes[i]

        # Calculate the influence spread of the seed node using the Linear Threshold Model
        spread = linear_threshold_model(G, [seed_node], p)

        # Add the influence spread to the list
        influence_spread.append(spread)

        # Print the results
        print("Community {}: Seed node: {}; Influence spread: {}".format(i, seed_node, spread))


    # Plot the graph with nodes colored by community and seed nodes highlighted
    colors = ['r' if node in communities[0] else 'b' for node in G.nodes()]
    seed_colors = ['#FF0000' if node in seed_nodes else '#FFFFFF' for node in G.nodes()]
    plt.figure(figsize=(10, 8))
    nx.draw_networkx(G, pos=nx.spring_layout(G), node_color=colors, node_size=400, linewidths=0.5, with_labels=True, font_size=10, font_weight='bold', edge_color='gray', alpha=0.9);
    nx.draw_networkx_nodes(G, pos=nx.spring_layout(G), nodelist=seed_nodes, node_color=seed_colors, node_size=400, linewidths=1, with_labels=True, font_size=10, font_weight='bold', edge_color='black')
    plt.show()

    

  # Define the node colors
    #node_colors = ['red' if node in seed_nodes else 'blue' for node in G.nodes()]

  # Define the edge colors
    #edge_colors = ['black' if edge in G.edges() else 'white' for edge in nx.non_edges(G)]

  # Create the Plotly figure
    #fig = go.Figure()

  # Add the nodes to the figure
    #pos = nx.spring_layout(G)

    #fig.add_trace(go.Scatter(x=[pos[node][0] for node in G.nodes()],
                         #y=[pos[node][1] for node in G.nodes()],
                         #mode='markers',
                         #marker=dict(color=node_colors),
                         #text=[f'Community: {i}, Influence Spread: {spread}' for i, spread in enumerate(influence_spread)],
                         #hoverinfo='text'))

  # Add the edges to the figure
    #for edge in G.edges():
      #fig.add_trace(go.Scatter(x=[pos[edge[0]][0], pos[edge[1]][0]],
                             #y=[pos[edge[0]][1], pos[edge[1]][1]],
                             #mode='lines',
                             #line=dict(color='black', width=1)))

  # Update the layout
    #fig.update_layout(title_text='Karate Club Network with Communities, Seed Nodes, and Influence Spread',
                  #showlegend=False,
                  #hovermode='closest')

  # Show the figure
    #fig.show()


if __name__ == '__main__':
    main()


TypeError: randint() missing 2 required positional arguments: 'a' and 'b'